In [17]:
import os
import json
import pandas as pd
import numpy as np
import tqdm
import scipy.sparse as sp
from matplotlib import pyplot as plt
import seaborn as sns
import mglearn
%matplotlib inline
from IPython.display import display
from pprint import pprint
pd.set_option('display.max_columns', 50)
DATA_PATH='D:\download'
t_e = pd.read_csv(
    os.path.join(DATA_PATH, 'transactions_MMtoregress.csv'),
    
)
t_e['DEV0']=t_e['DEV0'].astype(np.int16)
t_e['DEV1']=t_e['DEV1'].astype(np.int16)
t_e['DEV3']=t_e['DEV3'].astype(np.int16)
t_e['DEV4']=t_e['DEV4'].astype(np.int16)
t_e['DEV5']=t_e['DEV5'].astype(np.int16)
t_e['DEV6']=t_e['DEV6'].astype(np.int16)
t_e['feature_5_65']=t_e['feature_5_65'].astype(np.int16)
t_e['feature_5_00']=t_e['feature_5_00'].astype(np.int16)
t_e['feature_5_68']=t_e['feature_5_68'].astype(np.int16)
t_e['feature_5_59']=t_e['feature_5_59'].astype(np.int16)
t_e['feature_5_-1']=t_e['feature_5_-1'].astype(np.int16)
t_e['smallf3']= t_e['smallf3'].astype(np.int16)
t_e['middlef3']=t_e['middlef3'].astype(np.int16)
t_e['bigf3']=t_e['bigf3'].astype(np.int16)
t_e['CM_S']=t_e['CM_S'].astype(np.int16)
t_e['CM_P']=t_e['CM_P'].astype(np.int16)
t_e['CM_R']=t_e['CM_R'].astype(np.int16)
t_e['feature_5_44']=t_e['feature_5_44'].astype(np.int16)
t_e['feature_3']=t_e['feature_3'].astype(np.int16)
t_e['duration']=t_e['duration'].astype(np.int16)
t_e['element_uid']=t_e['element_uid'].astype(np.int32)
t_e['user_uid']=t_e['user_uid'].astype(np.int32)
t_e.info()
t_e=t_e.drop(['Unnamed: 0'],1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118932 entries, 0 to 118931
Data columns (total 31 columns):
Unnamed: 0        118932 non-null int64
element_uid       118932 non-null int32
user_uid          118932 non-null int32
watched_time_x    118736 non-null float64
rating            462 non-null float64
duration          118932 non-null int16
feature_2         118932 non-null float64
feature_3         118932 non-null int16
feature_4         118932 non-null float64
quantity          118932 non-null int64
rating_mean       114441 non-null float64
CM_S              118932 non-null int16
CM_P              118932 non-null int16
CM_R              118932 non-null int16
feature_5_65      118932 non-null int16
feature_5_00      118932 non-null int16
feature_5_68      118932 non-null int16
feature_5_59      118932 non-null int16
feature_5_-1      118932 non-null int16
feature_5_44      118932 non-null int16
smallf3           118932 non-null int16
middlef3          118932 non-null int16
bi

In [18]:
t_e.duration= t_e.duration.map(lambda x: 5 if x == 0 else x)
t_e.watched_time_x=(t_e.watched_time_y/60)/(t_e.duration)
t_e[t_e.watched_time_x > 50] = t_e[t_e.watched_time_x > 50].fillna(10.0)
t_e[t_e.watched_time_x > 25] = t_e[t_e.watched_time_x > 25].fillna(8.0)
t_e[t_e.watched_time_x > 15] = t_e[t_e.watched_time_x > 15].fillna(7.0)

In [19]:
t_e.watched_time_y=(t_e.watched_time_y/60)
t_e["user_rating_mean"] = t_e.user_uid.map(t_e.groupby("user_uid").rating.mean())
from collections import Counter
coun = pd.DataFrame.from_dict(Counter(t_e['user_uid']), orient='index')
coun['user_uid']=coun.index
coun['quantity_user_views']=coun[0]

coun=coun.reset_index()

coun=coun.drop(['index'], axis=1)
coun=coun.drop([0],1)
t_e=pd.merge(t_e, coun, on='user_uid')
t_e["wt_y_user_mean"] = t_e.user_uid.map(t_e.groupby("user_uid").watched_time_y.mean())
t_e["wt_x_user_mean"] = t_e.user_uid.map(t_e.groupby("user_uid").watched_time_x.mean())
t_e["new_content"]=t_e['ts-f1'].map(lambda x: 1 if x<31 else 0)

In [20]:

t_e['quartile__element_views'] = pd.qcut(t_e['quantity'], 4, ['1','2','3','4'])
t_e['quartile__duration'] = pd.qcut(t_e['duration'], 4, ['1','2','3','4'])
t_e['quartile__wtx'] = pd.qcut(t_e['wt_x_user_mean'], 4, ['1','2','3','4'])
t_e['quartile__f2'] = pd.qcut(t_e['feature_2'], 4, ['1','2','3','4'])
t_e['quartile__f4'] = pd.qcut(t_e['feature_4'], 4, ['1','2','3','4'])

t_e["Q_el_views=1"]=t_e['quartile__element_views'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_el_views=4"]=t_e['quartile__element_views'].map(lambda x: 1 if x=='4' else 0)
t_e["Q_duration=1"]=t_e['quartile__duration'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_duration=4"]=t_e['quartile__duration'].map(lambda x: 1 if x=='4' else 0)
t_e["Q_wtx=1"]=t_e['quartile__wtx'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_wtx=4"]=t_e['quartile__wtx'].map(lambda x: 1 if x=='4' else 0)

t_e["Q_f2=1"]=t_e['quartile__f2'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_f2=4"]=t_e['quartile__f2'].map(lambda x: 1 if x=='4' else 0)
t_e["Q_f4=1"]=t_e['quartile__f4'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_f4=4"]=t_e['quartile__f4'].map(lambda x: 1 if x=='4' else 0)
t_e=t_e.drop(['quartile__element_views',  'quartile__duration', 'quartile__wtx', 'quartile__f2', 'quartile__f4'],1)
t_e['new_content']=t_e['new_content'].astype(np.int16)
t_e['Q_el_views=1']=t_e['Q_el_views=1'].astype(np.int16)
t_e['Q_el_views=4']=t_e['Q_el_views=4'].astype(np.int16)
t_e['Q_duration=1']=t_e['Q_duration=1'].astype(np.int16)
t_e['Q_duration=4']=t_e['Q_duration=4'].astype(np.int16)
t_e['Q_wtx=1']=t_e['Q_wtx=1'].astype(np.int16)
t_e['Q_wtx=4']=t_e['Q_wtx=4'].astype(np.int16)
t_e['Q_f2=1']=t_e['Q_f2=1'].astype(np.int16)
t_e['Q_f2=4']=t_e['Q_f2=1'].astype(np.int16)
t_e['Q_f4=1']=t_e['Q_f4=1'].astype(np.int16)
t_e['Q_f4=4']=t_e['Q_f4=1'].astype(np.int16)
t_e.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118932 entries, 0 to 118931
Data columns (total 45 columns):
element_uid            118932 non-null int32
user_uid               118932 non-null int32
watched_time_x         118932 non-null float64
rating                 3126 non-null float64
duration               118932 non-null int64
feature_2              118932 non-null float64
feature_3              118932 non-null int16
feature_4              118932 non-null float64
quantity               118932 non-null int64
rating_mean            114752 non-null float64
CM_S                   118932 non-null int16
CM_P                   118932 non-null int16
CM_R                   118932 non-null int16
feature_5_65           118932 non-null int16
feature_5_00           118932 non-null int16
feature_5_68           118932 non-null int16
feature_5_59           118932 non-null int16
feature_5_-1           118932 non-null int16
feature_5_44           118932 non-null int16
smallf3                118

In [21]:
t_e.isnull().sum()/t_e.shape[0]

element_uid            0.000000
user_uid               0.000000
watched_time_x         0.000000
rating                 0.973716
duration               0.000000
feature_2              0.000000
feature_3              0.000000
feature_4              0.000000
quantity               0.000000
rating_mean            0.035146
CM_S                   0.000000
CM_P                   0.000000
CM_R                   0.000000
feature_5_65           0.000000
feature_5_00           0.000000
feature_5_68           0.000000
feature_5_59           0.000000
feature_5_-1           0.000000
feature_5_44           0.000000
smallf3                0.000000
middlef3               0.000000
bigf3                  0.000000
ts-f1                  0.000000
watched_time_y         0.000000
DEV0                   0.000000
DEV1                   0.000000
DEV3                   0.000000
DEV4                   0.000000
DEV5                   0.000000
DEV6                   0.000000
user_rating_mean       0.918777
quantity

In [22]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor


t_e=t_e.drop(['user_rating_mean'],1)
transactions_exp=t_e


transactions_exp.rating_mean=transactions_exp.rating_mean.fillna(5.0) 

transactions_exp['worst']=transactions_exp.watched_time_x.map(lambda x:1 if (x < 0.2 and x >0.005) else 0)
transactions_exp[transactions_exp.worst == 1]=transactions_exp[transactions_exp.worst == 1].fillna(2.0)
transactions_exp['worst1']=transactions_exp.watched_time_x.map(lambda x:1 if (x < 0.6 and x >0.19999) else 0)
transactions_exp[transactions_exp.worst1 == 1]=transactions_exp[transactions_exp.worst1 == 1].fillna(4.0)
transactions_exp=transactions_exp.drop(['worst', 'worst1'],1)



X_train = transactions_exp[~transactions_exp.rating.isnull()]
X_train=X_train.drop(['rating'],1)
X_test = transactions_exp[transactions_exp.rating.isnull()]
X_test=X_test.drop(['rating'],1)
y_train = transactions_exp[~transactions_exp.rating.isnull()].rating.astype(np.float)
y_test = transactions_exp[transactions_exp.rating.isnull()].rating
X_test.rating_mean=X_test.rating_mean.fillna(6.0) 
X_train.rating_mean=X_train.rating_mean.fillna(6.0)


model2 = RandomForestRegressor( n_estimators=522, max_depth=8, random_state=241, n_jobs=-1)

model2.fit(X_train, y_train)
submit1 = pd.DataFrame(model2.predict(X_test))

estimator=xgb.XGBRegressor(learning_rate=0.1, max_depth=8, n_estimators=700, n_jobs=-1  )

    
estimator.fit(X_train, y_train)
submit2 = pd.DataFrame(estimator.predict(X_test))

#kf = KFold(n_splits=5, shuffle=True, random_state=241) 


#model2 = RandomForestRegressor( n_estimators=522, max_depth=8, random_state=241, n_jobs=-1)
#score=cross_val_score(model2, X_train, y_train, cv=kf, n_jobs=-1, scoring='mean_squared_error')
#print("Score is", score.mean())
#

In [7]:
Score is -0.5044543186138928  без кучи новых признаков
Score is -0.5280732970557251 с кучей

SyntaxError: invalid syntax (<ipython-input-7-d3c8aecfed60>, line 1)

In [33]:
model2.fit(X_train, y_train)
submit1 = pd.DataFrame(model2.predict(X_test))


In [8]:

estimator=xgb.XGBRegressor(learning_rate=0.1, max_depth=8, n_estimators=700, n_jobs=-1  )

    
estimator.fit(X_train, y_train)
submit1 = pd.DataFrame(estimator.predict(X_test))

In [23]:

X_test=X_test.drop([
'watched_time_x',
'duration',
'feature_2',
'feature_3',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'watched_time_y',
'DEV0',
'DEV1',
'DEV3',
'DEV4',
'DEV5',
'DEV6',
'quantity_user_views',
'wt_y_user_mean',         
'wt_x_user_mean',
'new_content',
'Q_el_views=1',
'Q_el_views=4',
'Q_duration=1',
'Q_duration=4',
'Q_wtx=1',
'Q_wtx=4',
'Q_f2=1',
'Q_f2=4',
'Q_f4=1',
'Q_f4=4'],1)
#submit[0]=submit[0].round()
#submit[0]=submit[0].astype(np.int16)
X_test=X_test.reset_index()
X_test=X_test.drop(['index'], 1)

OLD=pd.concat([X_test, submit2], 1)
#OLD.rating=OLD.rating.round()
OLD['rating']=OLD[0]
OLD=OLD.drop([0],1)
#ratings=ratings.drop(['ts'],1)
X_train = transactions_exp[~transactions_exp.rating.isnull()]
#X_train.rating_mean=X_train.rating_mean.fillna(6.0)
X_train=X_train.drop([
'watched_time_x',
'duration',
'feature_2',
'feature_3',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'watched_time_y',
'DEV0',
'DEV1',
'DEV3',
'DEV4',
'DEV5',
'DEV6',
'quantity_user_views',
'wt_y_user_mean',         
'wt_x_user_mean',
'new_content',
'Q_el_views=1',
'Q_el_views=4',
'Q_duration=1',
'Q_duration=4',
'Q_wtx=1',
'Q_wtx=4',
'Q_f2=1',
'Q_f2=4',
'Q_f4=1',
'Q_f4=4'],1)
X_train=X_train.reset_index()
X_train=X_train.drop(['index'], 1)
OLD=OLD.drop(['user_rating_mean'],1)
OLD.to_csv("MultMovRatingXGB5.csv")


OLD=pd.concat([X_test, submit1], 1)
#OLD.rating=OLD.rating.round()
OLD['rating']=OLD[0]
OLD=OLD.drop([0],1)
X_train=X_train.reset_index()
X_train=X_train.drop(['index'], 1)
OLD=OLD.drop(['user_rating_mean'],1)

OLD.to_csv("MultMovRatingRF5.csv")


In [11]:
OLD=OLD.append(X_train)
OLD.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118932 entries, 0 to 26860
Data columns (total 4 columns):
element_uid         118932 non-null int32
rating              118932 non-null float64
user_rating_mean    118932 non-null float64
user_uid            118932 non-null int32
dtypes: float64(2), int32(2)
memory usage: 3.6 MB


In [12]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92071 entries, 0 to 92070
Data columns (total 3 columns):
element_uid         92071 non-null int32
user_uid            92071 non-null int32
user_rating_mean    92071 non-null float64
dtypes: float64(1), int32(2)
memory usage: 1.4 MB


In [13]:
OLD=OLD.drop(['user_rating_mean'],1)
OLD.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118932 entries, 0 to 26860
Data columns (total 3 columns):
element_uid    118932 non-null int32
rating         118932 non-null float64
user_uid       118932 non-null int32
dtypes: float64(1), int32(2)
memory usage: 2.7 MB


In [14]:
OLD.to_csv("MultMovRatingXGB.csv")

In [12]:
importances = model2.feature_importances_
indices = np.argsort(importances)[::-1]
feature_names=X_train.columns
print("Feature importances:")
for f, idx in enumerate(indices):
    print("{:2d}. feature '{:5}' ({:.4f})".format(f+1, feature_names[idx], importances[idx]))


Feature importances:


IndexError: index 29 is out of bounds for axis 0 with size 4